In [1]:
## Introduction

In [2]:
## Outline
# 1.  Data Ingestion & Wrangling
# 1a. County Wifi Locations dataset
# 1b. Public Schools Dataset
# 1c. Xfinity Wifi Locations
# 1d. MPCS School report

# 2. Exploratory Data Analysis 
# 2a. County Govermeny Wifi Locations
# 2b. Public Schools
# 1c. Xfinity Wifi Locations
# 1d. MPCS School report

# 3. Statistical Analysis

# 4. Mapping

In [3]:
## 1.  Data Ingestion & Wrangling

In [4]:
# importing the packages 
import pandas as pd
import numpy as np

In [5]:
# 1a. County Wifi Locations dataset

In [6]:
# reading the csv files for county government wifi locations
df = pd.read_csv('county_government_wifi_locations.csv')

In [7]:
# viewing the head of the dataframe
df.head()

,Agency,Department,Name,Address,Location
0,MCG,DPL,White Oak Library,"11701 New Hampshire Avenue, Silver Spring, MD ...",NaN
1,MCG,RSC,Sidney Kramer Upcounty Service ctr.,"12900 Middlebrook Road, Germantown MD",POINT (-77.267914 39.179332)
2,MCG,DTS,L3 Helpdesk & Office,"7361 Calhoun Plaza, Rockville, MD 20850",NaN
3,MCG,CAO/CEX/CC/Courts,OIG/ERP Office,"51 Monroe Street, Rockville, MD 20850",POINT (-90.409668 40.720439)
4,MCG,FRS,F&RS,"19801 Bealsville Road, Bealsville, MD 20839",NaN


In [8]:
# viewing the unique departments that the data contains
df.Department.unique()

array(['DPL', 'RSC', 'DTS', 'CAO/CEX/CC/Courts', 'FRS', 'REC', 'DOT',
       'DGS', 'HHS', 'BOE', 'DED/DEP', 'DOCR', 'MCPD', 'DLC',
       'PIO/PublicSafety', 'Warehouse', 'DHS', 'DPS'], dtype=object)

In [9]:
# Looking at how many are missing
# it is an issue that there is missing data because that will affect in plotting the points
df.Location.isnull().sum() 

34

In [10]:
# fixing the way that the "points" are in the dataframe
# socrata POINT datatypes list their coordinates as "longitude,latitude"
df[["Longitude", "Latitude"]] = df["Location"].str.replace("POINT \(","").str.replace("\)","").str.split(" ", expand=True)
print(df)

    Agency         Department                                 Name  \
0      MCG                DPL                    White Oak Library   
1      MCG                RSC  Sidney Kramer Upcounty Service ctr.   
2      MCG                DTS                 L3 Helpdesk & Office   
3      MCG  CAO/CEX/CC/Courts                       OIG/ERP Office   
4      MCG                FRS                                 F&RS   
..     ...                ...                                  ...   
173    MCG                DGS                   Swing Office Space   
174    MCG                HHS             HHS - 1301 Piccard Drive   
175    MCG                FRS              Wheaton Resource Center   
176    MCG                DOT     Colesville Highway Service Depot   
177    MCG                RSC             Bethesda/Chevy Chase RSC   

                                               Address  \
0    11701 New Hampshire Avenue, Silver Spring, MD ...   
1                12900 Middlebrook Road, Ge

In [11]:
# using geopy to fill in the missing data.
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="capstone")

In [12]:
# the NaN are floats
type(df['Location'][0])

float

In [13]:
# the values that are not missing in locations are strings
type(df['Location'][10])

str

In [14]:
# counting the number of times the geolocator fails
failed = 0 
# getting the value and index of the all the rows
for row_num, row in enumerate(df["Location"]):
  # if the value is NaN (float type) 
  if type(row) == float:
    # geocode the address at the same row
    location = geolocator.geocode(df["Address"][row_num])
    # if geocoded successfully
    if location is not None:
      df["Latitude"][row_num] = location.latitude
      df["Longitude"][row_num] = location.longitude
    else:
      failed += 1
      print(df["Address"][row_num])
print(failed)

7361 Calhoun Plaza,  Rockville, MD 20850
19801 Bealsville Road, Bealsville, MD 20839
8540 Anniversary Drive, Gaithersburg, MD 20877
14401 Connecticut Avenue, Silver Spring, MD 20906
8630 Fenton Street, 10th floor, Silver Spring, MD
1400 Spring St suite 305, Silver Spring, MD 20910
19300 Georgia Avenue, Brookville,  MD 20833
3300 Briggs Chaney Road, Silver Spring, MD 20904
14715 Good Hope Road, Silver Spring, MD 20905
13900 Old Columbia Pike, Silver Spring, MD 20904
14910 Old Comumbia Pike, Burtonsville, MD 20866
18905 Kingsview Drive, Germantown, MD, 20874
4407 Aspen Hill Road, Rockville, MD
14335 Cape May Road, Silver Spring, MD 20904
14


In [15]:
geolocator = Nominatim(user_agent="Your_Name")
location = geolocator.geocode('8630 Fenton Street')
print(location.address)
print((location.latitude, location.longitude))

GentleTouch Dental Care, 8630, Fenton Street, Blair Circle, Silver Spring, Montgomery County, Maryland, 20910, United States
(38.9979273, -77.0286378)


In [16]:
# 1b. Public Schools Dataset

In [17]:
# reading and viewing the head of the csv files for county government wifi locations
df2 = pd.read_csv('Public_Schools.csv')
df2.head()

,CATEGORY,SCHOOL NAME,ADDRESS,CITY,ZIP CODE,PHONE,URL,LONGITUDE,LATITUDE,LOCATION
0,ELEMENTARY SCHOOLS,Beall ES,451 Beall Ave,Rockville,20850,301-279-8460,http://www.montgomeryschoolsmd.org/schools/bea...,-77.161758,39.087775,"451 Beall Ave\nRockville, MD 20850\n(39.087775..."
1,ELEMENTARY SCHOOLS,Clearspring ES,9930 Moyer Rd,Damascus,20872,301-253-7004,http://www.montgomeryschoolsmd.org/schools/cle...,-77.207301,39.265783,"9930 Moyer Rd\nDamascus, MD 20872\n(39.2657831..."
2,ELEMENTARY SCHOOLS,Sherwood ES,1401 Olney Sandy Spring Rd,Sandy Spring,20860,301-924-3195,http://www.montgomeryschoolsmd.org/schools/she...,-77.033116,39.149584,"1401 Olney Sandy Spring Rd\nSandy Spring, MD 2..."
3,ELEMENTARY SCHOOLS,Pine Crest ES,201 Woodmoor Dr,Silver Spring,20901,301-649-8066,http://www.montgomeryschoolsmd.org/schools/pin...,-77.005038,39.021311,"201 Woodmoor Dr\nSilver Spring, MD 20901\n(39...."
4,MIDDLE SCHOOLS,Earle B. Wood MS,14615 Bauer Dr,Rockville,20852,301-460-2150,http://www.montgomeryschoolsmd.org/schools/woodms,-77.107529,39.093295,"14615 Bauer Dr\nRockville, MD 20852\n(39.09329..."


In [18]:
# Change the column names, lower case and removing spaces
df2.columns =['category', 'school_name', 'address', 'city', 'zip_code', 'phone','url','longitude','latitude','location']
df2.head()

,category,school_name,address,city,zip_code,phone,url,longitude,latitude,location
0,ELEMENTARY SCHOOLS,Beall ES,451 Beall Ave,Rockville,20850,301-279-8460,http://www.montgomeryschoolsmd.org/schools/bea...,-77.161758,39.087775,"451 Beall Ave\nRockville, MD 20850\n(39.087775..."
1,ELEMENTARY SCHOOLS,Clearspring ES,9930 Moyer Rd,Damascus,20872,301-253-7004,http://www.montgomeryschoolsmd.org/schools/cle...,-77.207301,39.265783,"9930 Moyer Rd\nDamascus, MD 20872\n(39.2657831..."
2,ELEMENTARY SCHOOLS,Sherwood ES,1401 Olney Sandy Spring Rd,Sandy Spring,20860,301-924-3195,http://www.montgomeryschoolsmd.org/schools/she...,-77.033116,39.149584,"1401 Olney Sandy Spring Rd\nSandy Spring, MD 2..."
3,ELEMENTARY SCHOOLS,Pine Crest ES,201 Woodmoor Dr,Silver Spring,20901,301-649-8066,http://www.montgomeryschoolsmd.org/schools/pin...,-77.005038,39.021311,"201 Woodmoor Dr\nSilver Spring, MD 20901\n(39...."
4,MIDDLE SCHOOLS,Earle B. Wood MS,14615 Bauer Dr,Rockville,20852,301-460-2150,http://www.montgomeryschoolsmd.org/schools/woodms,-77.107529,39.093295,"14615 Bauer Dr\nRockville, MD 20852\n(39.09329..."


In [19]:
# changing the lables of the departments

In [20]:
# checking that there is no null in longitude and latitde
df2.longitude.isnull().sum()
df2.latitude.isnull().sum()

0

In [21]:
# Changing the school names to prepare to merge with other school dataset

In [22]:
# changing index cols with rename()

df2['school_name'] = df2['school_name'].str.replace('ES', 'Elementary')
df2['school_name'] = df2['school_name'].str.replace('MS', 'Middle')
df2['school_name'] = df2['school_name'].str.replace('HS', 'High')

In [23]:
df2.head()

,category,school_name,address,city,zip_code,phone,url,longitude,latitude,location
0,ELEMENTARY SCHOOLS,Beall Elementary,451 Beall Ave,Rockville,20850,301-279-8460,http://www.montgomeryschoolsmd.org/schools/bea...,-77.161758,39.087775,"451 Beall Ave\nRockville, MD 20850\n(39.087775..."
1,ELEMENTARY SCHOOLS,Clearspring Elementary,9930 Moyer Rd,Damascus,20872,301-253-7004,http://www.montgomeryschoolsmd.org/schools/cle...,-77.207301,39.265783,"9930 Moyer Rd\nDamascus, MD 20872\n(39.2657831..."
2,ELEMENTARY SCHOOLS,Sherwood Elementary,1401 Olney Sandy Spring Rd,Sandy Spring,20860,301-924-3195,http://www.montgomeryschoolsmd.org/schools/she...,-77.033116,39.149584,"1401 Olney Sandy Spring Rd\nSandy Spring, MD 2..."
3,ELEMENTARY SCHOOLS,Pine Crest Elementary,201 Woodmoor Dr,Silver Spring,20901,301-649-8066,http://www.montgomeryschoolsmd.org/schools/pin...,-77.005038,39.021311,"201 Woodmoor Dr\nSilver Spring, MD 20901\n(39...."
4,MIDDLE SCHOOLS,Earle B. Wood Middle,14615 Bauer Dr,Rockville,20852,301-460-2150,http://www.montgomeryschoolsmd.org/schools/woodms,-77.107529,39.093295,"14615 Bauer Dr\nRockville, MD 20852\n(39.09329..."


In [24]:
demo = pd.read_csv('mcps_demo.csv',sep=',')

In [25]:
demo.head()

,Unnamed: 0,Name,Total Enrollment,Free lunch eligible1,Reduced-price lunch eligible1,Directly certified2
0,0,A. Mario Loiederman Middle,999,424,124,127
1,1,Albert Einstein High,"1,818",492,153,145
2,2,Alternative Programs,126,58,2,40
3,3,Arcola Elementary,748,456,95,159
4,4,Argyle Middle,"1,024",444,130,131


In [34]:
demo.columns =['number', 'school_name', 'total_enrollment', 'free_lunch', 'reduced_lunch', 'directly_certified']
demo.head()

,number,school_name,total_enrollment,free_lunch,reduced_lunch,directly_certified
0,0,A. Mario Loiederman Middle,999,424,124,127
1,1,Albert Einstein High,"1,818",492,153,145
2,2,Alternative Programs,126,58,2,40
3,3,Arcola Elementary,748,456,95,159
4,4,Argyle Middle,"1,024",444,130,131


In [26]:
#df3 = pd.concat([df2, demo])

In [36]:
df3 = pd.merge(df2, demo[['total_enrollment']],
               on='school_name')
df3.head()

KeyError: 'school_name'

In [ ]:
df3.head()

In [ ]:
## 2. Exploratory Data Analysis 

In [ ]:
# import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 2a. County Govermeny Wifi Locations

In [ ]:
sns.set_theme(style="darkgrid")
sns.countplot(y='Department',data=df)
plt.xlabel('Types of Departments')
plt.ylabel('Count')
plt.title('Departments')
plt.show()

In [ ]:
# 2b. Public Schools

In [ ]:
# graphing the number of categories of schools
sns.set_theme(style="darkgrid")
sns.countplot(y='category',data=df2)
plt.xlabel('Types of Schools')
plt.ylabel('Count')
plt.title('School')
plt.show()

In [ ]:
sns.set_theme(style="darkgrid")
sns.countplot(y='city',data=df2)
plt.xlabel('County')
plt.ylabel('City')
plt.title('Cities of MCPS Schools')
plt.show()